# GMSO - General Molecular Simulation Object

## Working with GMSO base and core classes

[GMSO](https://github.com/mosdef-hub/gmso) allows for engine agnostic python object to store everything needed for writing molecule input files for simulation.


This notebook is designed to provide advanced principles for interfacing with GMSO. Tutorials get more in-depth in terms of complex usage denoted by the `beginners`, `intermediates`, and `experts` tags on these notebooks. For a full list of the examples discussed in these tutorials, please see [the GMSO Tutorials README](README.md).

After completing this notebook, move on to [this notebook](06_for_experts.ipynb) to learn about how to fix issues during atomtyping.

Contained in this notebook is examples of:
* Base classes
* Core classes



## Base classes (module gmso.abc)
This module provides the abstract base classes for all other core data structures used in gmso. Our abstract base classes inherit from [pydantic](https://pydantic-docs.helpmanual.io/)'s `BaseModel` class which provides type hints as well as runtime data validation together with out-of-the-box serialization. The module structure is as follows:
```
gmso/abc 
├── abstract_connection.py 
├── abstract_potential.py 
├── abstract_site.py 
├── gmso_base.py 
```


1. [`gmso_base.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/abc/gmso_base.py): Defines the class `GMSOBase` i.e. The base class for all our other classes that tweaks pydantic's `BaseModel` class to provide an `id`-based hasing as well as injects numpydoc style docstrings from the fields of the class.
---
2. [`abstract_site.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/abc/abstract_site.py): Defines the `Site` class which provides a basic topology site with following attributes: (a.) name (b.) position (c.) label
---
3. [`abstract_potential.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/abc/abstract_potential.py): Defines the abstract `Potential` class which is the base class for our `ParametricPotentials` as well as `PotentialTemplates`.
---
4. [`abstract_connection.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/abc/abstract_site.py): Defines the abstract `Connection` class which is the base class for our `Bond`, `Angle`, `Dihedral` and `Improper` classes.

### Extensibility 
The implementation of abstract base classes allows for a consistent and systematic expansion of the library. The `gmso.abc` would allow new data classes to be implemented as needs arise with built-in common attributes/methods and type-check. 

### Example: Implementing a Bead

The `Bead` class can now be implemented as a subclass of the abstract `Site` class. We can use the existing attributes from the super class like `name`, `position` etc... and define new attributes and methods for `Bead`. The goal is the consolidation of as many universal characteristics of a generic topology site into a base class (`Site`) and tweak its down-stream usage according to the needs of a particular site (like an `Atom` or a `Bead`). Usage of `Site` to create a `Bead` class is shown below:

In [ ]:
import warnings
warnings.simplefilter('ignore')
import unyt as u
from pydantic import Field, ValidationError

from gmso.abc.abstract_site import Site


class Bead(Site):
    __base_doc__ = "Basic Bead class inheriting from the Site Class"
    mass_: u.unyt_quantity = Field(
        default=1.0*u.amu,
        description='Mass of the bead'
    )
        
    charge_: u.unyt_quantity = Field(
        default=0.0*u.elementary_charge,
        description='Charge of the bead'
    )
    
    class Config:
        fields = {
            'mass_': 'mass',
            'charge_': 'charge'
        }
        alias_to_fields = {
            'mass': 'mass_',
            'charge': 'charge_'
        }
    
my_bead = Bead()
# When you inherit, the attribute(field) `name` is injected as the class name(Bead in this case)
print(f"Injected name is class name: {my_bead.name}")  

# We use pydantic for validation as well, for example if you assign a string to charge by accident :)
try:
    my_bead.charge = 'Some weird charge string'
except ValidationError as e:
    print(e)

In [ ]:
# Documentation is injected automatically as well
%pdoc Bead

## Core classes (module gmso.core)
In `gmso` we define the following core classes. All of our core classes make use of `gmso.abc` to define a particular site, connection or potential. The module `gmso.core`'s structure is as follows:

```
gmso/core/
├── angle.py
├── angle_type.py
├── atom.py
├── atom_type.py
├── bond.py
├── bond_type.py
├── box.py
├── dihedral.py
├── dihedral_type.py
├── element.py
├── forcefield.py
├── improper.py
├── improper_type.py
├── parametric_potential.py
└── topology.py
```

### Sites
1. [`atom.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/atom.py): Defines the class `gmso.core.atom.Atom` which inherits from `gmso.abc.abstract_site.Site` to define an `Atom`.

In [ ]:
from gmso.core.atom import Atom
atom1 = Atom(name='atom1', charge=2.0*u.elementary_charge)

# Dumping the model as json is easy
atom1.model_dump()


### Connections
1. [`bond.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/bond.py): Defines the class `gmso.core.bond.Bond` which inherits from `gmso.abc.abstract_connection.Connection` to define a 2-partner connection between `Atoms`.
---
2. [`angle.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/angle.py): Defines the class `gmso.core.angle.Angle` which inherits from `gmso.abc.abstract_connection.Connection` to define a 3-partner connection between `Atoms`.
---
3. [`dihedral.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/dihedral.py): Defines the class `gmso.core.dihedral.Dihedral` which inherits from `gmso.abc.abstract_connection.Connection` to define a 4-partner connection(dihedral) between `Atoms`.
---
4. [`improper.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/improper.py): Defines the class `gmso.core.improper.Improper` which inherits from `gmso.abc.abstract_connection.Connection` to define a 4-partner connection(improper) between `Atoms`.

In [ ]:
from gmso import Bond
atom2 = Atom(name='atom2', charge=-2.0*u.elementary_charge)
bond = Bond(connection_members=[atom1, atom2])
bond.connection_members


### Potentials
1. [`parametric_potential.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/parametric_potential.py): Defines the class `gmso.core.parametric_potential.ParametricPotential` which inherits from `gmso.abc.abstract_potential.Potential` to define a `ParametricPotential` class which stores the functional form of a Potential as a `sympy` expression and parameters of the potential expression as `unyt_quantities`.
---
2. [`atom_type.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/atom_type.py): Defines the class `gmso.core.atom_type.AtomType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for an `AtomType`.   
---
3. [`bond_type.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/bond_type.py): Defines the class `gmso.core.bond_type.BondType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for a `BondType`.
---
4. [`angle_type.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/angle_type.py): Defines the class `gmso.core.angle_type.AngleType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for an `AngleType`.
---
5. [`dihedral_type.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/dihedral_type.py) and [`improper_type.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/atom_type.py): Defines the classes `gmso.core.atom_type.DihedralType` and `gmso.core.improper_type.ImproperType` which inherit from `gmso.core.parametric_potential.ParametricPotential` which describe properties for a `DihedralType` and `ImproperType` respectively.

In [ ]:
from gmso.core.parametric_potential import ParametricPotential

# Handle potential expression using separate Expression class
new_potential = ParametricPotential(
    name='mypotential',
    expression='a*x+b',
    parameters={
        'a': 1.0*u.g,
        'b': 1.0*u.m
    },
    independent_variables={'x'}
)

try:
    new_potential.independent_variables = 'y'
except ValueError as e:
    print(e)

In [ ]:
from gmso import Topology, Atom, Bond
methane_top = Topology()
c = Atom(name='c')
h1 = Atom(name='h1')
h2 = Atom(name='h2')
h3 = Atom(name='h3')
h4 = Atom(name='h4')
ch1 = Bond(connection_members=[c,h1])
ch2 = Bond(connection_members=[c,h2])
ch3 = Bond(connection_members=[c,h3])
ch4 = Bond(connection_members=[c,h4])
methane_top.add_site(c, update_types=False)
methane_top.add_site(h1, update_types=False)
methane_top.add_site(h2, update_types=False)
methane_top.add_site(h3, update_types=False)
methane_top.add_site(h4, update_types=False)
methane_top.add_connection(ch1, update_types=False)
methane_top.add_connection(ch2, update_types=False)
methane_top.add_connection(ch3, update_types=False)
methane_top.add_connection(ch4, update_types=False)

# You can now infer angles, dihedrals and impropers from bonds
print(methane_top.n_angles)  # No angles now
methane_top.identify_connections() # Uses networkx to identify connections using subgraph isomorphism
print(methane_top.n_angles) 

### Forcefield
1. [`forcefield.py`](https://github.com/mosdef-hub/gmso/blob/main/gmso/core/forcefield.py): Defines the class `gmso.core.forcefield.ForceField` which defines the in memory representation of the gmso forcefield schema. An example forcefield XML (`tip3p.xml`) is shown below:

In [ ]:
from gmso import ForceField 
from gmso.tests.utils import get_path
tip3p_ff = ForceField(get_path('tip3p.xml'))
tip3p_ff.bond_types